<figure>
  <IMG SRC="https://raw.githubusercontent.com/fmeer/public-files/main/TUlogo.png" WIDTH=200 ALIGN="right">
</figure>

# CIEM5110 Workshop 1: Introduction to pyJive

## Preliminaries

To start working, we add the main pyJive folder to `path` and make some imports. Notice how we access nested pyJive components:

In [ ]:
import sys
sys.path.append('../../')

import matplotlib.pyplot as plt

from utils import proputils as pu
import main
from names import GlobNames as gn

## Running a simple bar model

To start off, we run the following simple bar problem:

<center>
  <figure>
    <IMG SRC="https://raw.githubusercontent.com/ibcmrocha/public/main/3elmbar.png" WIDTH=300>
  </figure>
</center>
    
We have a `bar.pro` file ready that describes this model. Take a look at it and try to see how things are being set up.
    
We now use pyJive to run it as is:

In [ ]:
props = pu.parse_file('bar.pro')

globdat = main.jive(props)

We can then use `globdat` to interact with model results. For instance, we can look at the nodal displacement values:

In [ ]:
print(globdat[gn.STATE0])

Do these values make sense? Check `bar.pro` and `bar.mesh` to see which values are used and try to derive a solution for this problem by hand.

## Changing the model on the fly

We can look at what `parse_file` read from `bar.pro`, make modifications and run the model again:

In [ ]:
props['model']['bar']['EA'] = 1000

globdat = main.jive(props)

print(globdat[gn.STATE0])

Do these new results make sense? **Now try this yourself**:
- Change the Dirichlet boundary condition on the left from $0$ to $1$
- Change the Neumann boundary condition on the right from $1$ to $10$

and rerun the model. Use the code block below for this:

We can make use of `DofSpace` to directly look at the displacement at a certain node, e.g. at the right-end node:

In [ ]:
node = globdat[gn.NGROUPS]['right'][0]
dof = globdat[gn.DOFSPACE].get_dof(node,'dx')

print('Displacement of node',node,'with DOF index',dof,':',globdat[gn.STATE0][dof])

## Using OOP to our advantage

With the object-oriented structure of pyJive, we can use inheritance to extend functionality with minimum effort. Let us create a new `BarModel` that does some extra printing for us:

In [ ]:
from models.barmodel import *

class MyBarModel(BarModel):
    def _get_matrix(self, params, globdat):
        D = np.array([[self._EA]])
        for elem in self._elems:
            # Get the nodal coordinates of each element
            inodes = elem.get_nodes()
            idofs = globdat[gn.DOFSPACE].get_dofs(inodes, DOFTYPES[0:self._rank])
            coords = np.stack([globdat[gn.NSET][i].get_coords() for i in inodes], axis=1)[0:self._rank, :]

            # Get the shape functions, gradients and weights of each integration point
            sfuncs = self._shape.get_shape_functions()
            grads, weights = self._shape.get_shape_gradients(coords)

            # Reset the element stiffness matrix
            elmat = np.zeros((self._dofcount, self._dofcount))
            
            for ip in range(self._ipcount):
                # Get the B matrix for each integration point
                B = np.zeros((1, self._nodecount))
                B[0, :] = grads[:, :, ip].transpose()

                # Compute the element stiffness matrix
                elmat += weights[ip] * np.matmul(np.transpose(B), np.matmul(D, B))

            # Add the element stiffness matrix to the global stiffness matrix
            params[pn.MATRIX0][np.ix_(idofs, idofs)] += elmat
            print('\nElement with nodes',inodes,'B matrix',B,'\n Stiffness matrix:\n',elmat,'\n')
          
            plt.imshow(params[pn.MATRIX0],vmin=-1.,vmax=2.,origin='upper')
            plt.xticks([0, 1, 2, 3])
            plt.yticks([0, 1, 2, 3])
            plt.show()
            plt.figure()           

We now tell `props` that we want to use the new model:

In [ ]:
props = pu.parse_file('bar.pro')
props['model']['bar']['type'] = 'MyBar'

We need to take one more step to let the program know about the existence of the `MyBarModel` we just created. In order to be able to add new models without having to touch the existing code anywhere, we make use of the so-called **factory** pattern. The factory pattern allows for using a string as input to define which object needs to be created. For this to work, we do need to tell the factory which types exist. For the existing models, modules and element types this is done in the `declare_models`, `declare_modules` and `declare_shapes` functions at the beginning of `main.jive`. If we pre-define the factory in the notebook, we can declare the model that we have just created to the factory and pass this on to `main.jive`. 

In [ ]:
import declare

globdat = {}

# default declaration

declare.declare_models(globdat)
declare.declare_modules(globdat)
declare.declare_shapes(globdat)
    
# Additional declaration of model defined in the notebook
    
factory = globdat[gn.MODELFACTORY]
factory.declare_model('MyBar',MyBarModel)

Finally, we call Jive. Keep an eye on the output!

In [ ]:
globdat = main.jive(props, globdat)

## Experimenting with models

Let us now build a very simple new model to understand how `models` and `modules` interact during a simulation.

**Do this yourself**: Use the shell below to print every action that reaches this model.

Note that our new model has no other functionality. But that agrees with the idea behind `take_action`: all models see all action calls and should ignore the ones it cannot answer.

In [ ]:
from models.model import Model

class PrintModel (Model):
    def take_action (self,action,params,globdat):
        # print here which action is being called and which
        # params are being given
        pass

We now add this model to our list of models, declare it and re-run the simulation:

In [ ]:
props = pu.parse_file('bar.pro')

props['model']['models'] = ['bar','diri','neum','print']
props['model']['print'] = {}
props['model']['print']['type'] = 'Print'

globdat = {}

# default declaration

declare.declare_models(globdat)
declare.declare_modules(globdat)
declare.declare_shapes(globdat)
    
# Additional declaration of the new model
    
factory = globdat[gn.MODELFACTORY]
factory.declare_model('Print',PrintModel)

# Run the simulation again

globdat = main.jive(props, globdat)